In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings

warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.set_option('float_format', '{:.2f}'.format)

In [2]:
current_date = dt.date.today()
day = current_date.day
month = current_date.month
year = current_date.year

way = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Илларионов\Выгрузка Еком\\'

In [3]:
data_IM = pd.read_excel(rf'{way}\{month}.{year}_продажи еком.xlsx')

In [4]:
# смотрим продажи с последнего мониторинга
data = dt.datetime.isoweekday(dt.date.today())
if data == 1:
    last_monitoring = (dt.date.today() - dt.timedelta(days=3)).strftime('%Y-%m-%d')
else:
    last_monitoring = (dt.date.today() - dt.timedelta(days=2)).strftime('%Y-%m-%d')
    
data_IM = data_IM[data_IM['created'] >= f'{last_monitoring}']

In [5]:
data_IM.sample(5)

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana
6321,5d9d130d-378c-40ca-b34c-03a7941f56ff,2024-05-28 08:26:17.648,Золотой пирсинг,done,79500858559.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ0786109,3-09-024,0.19,0.19,NaN,585,ИФ ПИРСИНГ В НОС,ПИРСИНГ,КРАСНЫЙ,БК СЕРЬГИ,SOKOLOV,60053,2405280826814,1937.00,MOBILE,2024-05-28 08:28:35.485,NaT,1910800,719.50,2078608659119,3-09-024,387,1.00,387.00,1550.00
11970,b0781840-65b7-4b58-a829-dbb7053aa680,2024-05-28 05:37:13.931,Золотая подвеска с топазом,done,79535985975.00,NaN,done,PICKUP_NOW,CASHIER,NaN,CASHIER,[CASHIER] CASHIER - CASHIER,ТОВ1179684,3-03-043,0.64,0.48,NaN,585,ДК С ОДНИМ КАМНЕМ,ПОДВЕСКА,КРАСНЫЙ,ПДК ПОДВЕСКИ,NaN,3013004ТГ#ТОП,2405280537945,5954.00,MOBILE,2024-05-28 05:49:14.705,NaT,1517069,1762.07,2078609522924,3-03-043,0,0.00,0.00,5954.00
17042,f8415964-2482-490f-82c9-6d0d675656ac,2024-05-28 12:12:36.966,Серебряные серьги с фианитами и родированием,done,79530352925.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - APP,ТОВ1272313,4-05-054,1.53,1.45,NaN,925,СИ ПУССЕТЫ,СЕРЬГИ,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ СЕРЬГИ РАЗН,NaN,12-1011,2405281212715,992.00,MOBILE,2024-05-28 12:46:01.021,NaT,1830081,369.42,2078608830202,4-05-054,319,0.62,197.78,794.22
15899,e7cd6935-5b93-4192-b199-cd33ab02d38d,2024-05-28 10:13:38.899,Золотой крест,done,79180908639.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1273856,6-04-050,0.59,0.59,NaN,375,ИФ КРЕСТ,ПОДВЕС КУЛЬТОВЫЙ,КРАСНЫЙ С БЕЛЫМ,БК ПОДВЕС КУЛЬТ,NaN,51-43-0000-28661#375,2405281013414,5362.00,MOBILE,2024-05-28 10:44:34.080,NaT,1675938,1330.69,2078609994225,6-04-050,0,0.00,0.00,5362.00
9248,8978a5d3-f153-4748-8461-fc2df93c3d48,2024-05-29 12:00:31.818,Золотая подвеска Крест с фианитами,done,79535544054.00,NaN,done,PICKUP_NOW,CASHIER,NaN,CASHIER,[CASHIER] CASHIER - CASHIER,ТОВ1234952,2-18-016,0.54,0.51,NaN,585,ИФ КРЕСТ,ПОДВЕС КУЛЬТОВЫЙ,КРАСНЫЙ,ИФ ПОДВЕС КУЛЬТ,NaN,31-2349-11-01,2405291200966,3240.00,MOBILE,2024-05-29 12:55:43.092,NaT,1081593,1424.34,2078585123580,2-18-016,0,0.00,0.00,3240.00


In [6]:
data_IM = data_IM.drop_duplicates()
data_IM = data_IM.reset_index(drop=True)

data_IM = data_IM[(data_IM['state_order'] != 'returned') & (data_IM['state_order'] != 'canceled')]
data_IM = data_IM[~((data_IM['delivery_type'] == 'PICKUP_NOW') & (data_IM['payment_type'] == 'CASHIER') & (data_IM['type_ibox'] == 'CASHIER'))|
                ((data_IM['delivery_type'] == 'PICKUP') & (data_IM['payment_type'] == 'CASHIER') & (data_IM['type_ibox'] == 'CASHIER'))]

In [7]:
data_IM.sample(5)

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana
61,19af4537-e462-4aec-bf50-651992ce0006,2024-05-28 13:44:23.965,Золотые серьги,done,79130057569.00,hgjktnghghjd@mail.com,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ1273016,3-05-120,1.34,1.34,NaN,585,ИФ БРЕНД,СЕРЬГИ,КРАСНЫЙ,БК СЕРЬГИ,РЕПЛИКИ,12-00-0000-11106,2405281344488,9217.00,MOBILE,2024-05-29 09:10:53.191,NaT,1145033,3115.96,2078603153876,3-05-120,1843,1.00,1843.00,7374.00
18,0561dac6-bffd-4b5d-9e6a-13db2e36bb08,2024-05-29 10:02:26.076,Серебряное кольцо с фианитами и родированием,done,79243744104.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - URL,ТОВ1315660,3-09-017,1.69,1.23,NaN,925,СИ ПРОЧЕЕ,КОЛЬЦО,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ КОЛЬЦО ИФ,NaN,1-10300,2405291002133,790.00,MOBILE,2024-05-29 11:20:45.643,NaT,1605232,399.49,2078608552139,3-09-017,0,0.00,0.00,790.00
10,03a8a342-4b2e-4735-99e8-e04be97cac70,2024-05-28 14:08:57.998,Серьги SOKOLOV из серебра с фианитами,done,79270348634.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_PAY,Тинькофф Долями,ТОВ0981263,2-18-011,2.43,1.98,NaN,925,СИ ПРОЧЕЕ,СЕРЬГИ,БЕЛЫЙ С РОДИРОВАНИЕМ,СИ СЕРЬГИ ИФ,SOKOLOV,94022291,2405281408480,2180.00,MOBILE,2024-05-28 14:16:33.250,NaT,2359704,489.60,2078525395053,2-18-011,0,0.00,0.00,2180.00
326,89b5dcae-323e-4d6b-927d-62a6098fce91,2024-05-29 08:34:15.778,Золотой крест,done,79009299053.00,lenalol098@icloud.com,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ1169516,2-13-015,0.73,0.73,NaN,375,ИФ КРЕСТ,ПОДВЕС КУЛЬТОВЫЙ,КРАСНЫЙ,БК ПОДВЕС КУЛЬТ,ADRIATICA,35-0494-30-00#375,2405290834733,4037.15,MOBILE,2024-05-29 10:26:45.953,NaT,8365738,1572.08,2078609033464,2-13-015,807,1.00,807.00,3230.15
168,4e8ea4b1-dc3b-4f66-a950-8cd47de4e2d4,2024-05-28 08:34:02.836,Золотая пустотелая цепочка Нонна с алмазной гр...,done,79284420222.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ1163905,6-04-024,1.92,1.92,NaN,585,НОННА ПУСТ,ЦЕПЬ,КРАСНЫЙ,ЦБ ЦЕПИ,NaN,31-01-0040-30200,2405280834481,13110.00,MOBILE,2024-05-28 09:21:46.748,NaT,8327924,4903.54,2078542991405,6-04-024,0,0.00,0.00,13110.00


In [8]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'], format = '%Y-%m-%d')

In [9]:
def str_float(row):
    row = str(row)
    row = row.replace(',','.')
    return row

data_IM['price'] = data_IM['price'].apply(str_float)
data_IM['price'] = data_IM['price'].astype(float)

data_IM['avg_net_weight'] = data_IM['avg_net_weight'].apply(str_float)
data_IM['avg_net_weight'] = data_IM['avg_net_weight'].astype(float)

In [10]:
data_IM = data_IM[data_IM['price'] > 1].reset_index(drop=True)
data_IM = data_IM[data_IM['avg_net_weight'] > 0].reset_index(drop=True)

In [11]:
data_IM['Товарное направление'] = np.where(data_IM['tg'].isin(['БК ПОДВЕС КУЛЬТ', 'БК СЕРЬГИ', 'БК КОЛЬЦА','БК КОЛЬЦА ОБРУЧ','БК ПОДВЕС ДЕКОР',  'БК ПЕЧАТКИ','БК АКСЕССУАРЫ','ОПТ БК']), 'БК',
                                np.where(data_IM['tg'].isin(['ПДК СЕРЬГИ', 'ПДК КОЛЬЦА','ПДК ПОДВЕСКИ']), 'ПДК',
                                np.where(data_IM['tg'].isin(['ДК СЕРЬГИ',  'ДК КОЛЬЦА',    'ДК ПОДВЕСКИ',    'ДК КОЛЬЦА ОБРУЧ']), 'ДК',
                                np.where(data_IM['tg'].isin(['ИФ СЕРЬГИ',  'ИФ КОЛЬЦА', 'ИФ ПОДВЕС ДЕКОР', 'ИФ ПЕЧАТКИ', 'ИФ ПОДВЕС КУЛЬТ','ИФ КОЛЬЦА ОБРУЧ','ИФ АКСЕССУАРЫ']), 'ИФ',
                                np.where(data_IM['tg'].isin(['ЦБ ЦЕПИ', 'ЦБ БРАСЛЕТЫ']), 'ЦБ',
                                np.where(data_IM['tg'].isin(['СИ ПОДВЕС БК','СИ КОЛЬЦО ИФ','СИ БРАСЛЕТ БК','СИ КОЛЬЦО БК','СИ ЦЕПЬ','СИ СЕРЬГИ БК','СИ СЕРЬГИ ИФ','СИ ПОДВЕС ИФ','СИ СЕРЬГИ РАЗН','СИ СЕРЬГИ ЦВ/К',\
                                                            'СИ КОЛЬЦО ЦВ/К','','СИ ШАРМЫ','СИ ОБРУЧАЛЬНОЕ','СИ КОЛЬЦО ПЕЧАТ','СИ ПОДВЕСЦВ/К','СИ БРАСЛЕТ СК','СИ ЖЕМЧУГ',\
                                                            'СИ АКСЕССУАР','СИ ДФРС ПРОЕКТ','СИ КЕРАМИКА','СИ ПОСУДА','СИ ЯНТАРЬ','СИ ЭМАЛЬ','СИ ВЫРИЦА']), 'СИ',
                                np.where(data_IM['tg'].isin(['ЧАСЫ']), 'ЧАСЫ','другое')))))))

data_IM['Товарное направление'].value_counts()

Товарное направление
СИ        155
БК        111
ИФ         96
ЦБ         75
ДК         53
ПДК        26
другое     23
Name: count, dtype: int64

In [12]:
proverka = data_IM[data_IM['Товарное направление']=='другое']
proverka['tg'].unique()

array(['СИ СЕРЬГИ\xa0 ЦВ/К', 'ОПТ ИФ', 'СИ КОЛЬЕ СК', 'ВЫРИЦА СЕРЕБРО',
       'СИ ПОДВЕС\xa0ЦВ/К', 'СИ БРАСЛЕТ ЦВ/К', 'СИ БРАСЛЕТ БК Декор'],
      dtype=object)

In [13]:
data_IM = data_IM[~(data_IM['name'].str.contains('каучук')
        |data_IM['name'].str.contains('шнурок')
        |data_IM['name'].str.contains('текстиль')
        |data_IM['name'].str.contains('хлопок')
        |data_IM['name'].str.contains('шелк')
        |data_IM['name'].str.contains('на нитке')
        |data_IM['name'].str.contains('кожа')
        |data_IM['name'].str.contains('жемчуг'))]

In [14]:
data_IM.info()

<class 'pandas.core.frame.DataFrame'>
Index: 522 entries, 0 to 538
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         522 non-null    object        
 1   created                    522 non-null    datetime64[ns]
 2   name                       522 non-null    object        
 3   state_order                522 non-null    object        
 4   recipient_phone            522 non-null    float64       
 5   recipient_email            164 non-null    object        
 6   state_item                 522 non-null    object        
 7   delivery_type              522 non-null    object        
 8   payment_type               522 non-null    object        
 9   employee_id                39 non-null     object        
 10  type_ibox                  522 non-null    object        
 11  type_name                  522 non-null    object        
 12  tov          

In [15]:
data_IM = data_IM[(data_IM['Товарное направление'].isin(['БК', 'ЦБ']))&(data_IM['proba'] == '585')]

In [16]:
data_IM['Товарное направление'].value_counts()

Товарное направление
БК    91
ЦБ    62
Name: count, dtype: int64

In [17]:
data_IM['proba'].unique()

array(['585'], dtype=object)

In [18]:
data_IM['date_status_order_done'].value_counts()

date_status_order_done
2024-05-29 09:40:43.794    1
2024-05-28 16:28:42.670    1
2024-05-28 11:29:58.208    1
2024-05-28 10:06:51.240    1
2024-05-29 14:01:05.771    1
                          ..
2024-05-29 09:50:17.122    1
2024-05-29 16:04:32.849    1
2024-05-29 11:36:11.965    1
2024-05-29 09:49:14.348    1
2024-05-29 11:47:19.882    1
Name: count, Length: 153, dtype: int64

In [19]:
previous_date = data_IM['date_status_order_done'].min()

In [20]:
filtr_date = data_IM['date_status_order_done'].max()
print(filtr_date)

2024-05-29 18:13:56.083000


In [21]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'], format='%Y-%m-%d')

In [22]:
data_IM['date_status_order_done'] = pd.to_datetime(data_IM['date_status_order_done'].apply(lambda x: x.strftime("%Y-%m-%d")))

In [23]:
data_today = data_IM.query("date_status_order_done >= @previous_date")
data_today.head()

,id,created,name,state_order,recipient_phone,recipient_email,state_item,delivery_type,payment_type,employee_id,type_ibox,type_name,tov,kbk_recieve_delivery,avg_weight,avg_net_weight,materials,proba,design,type1,type3,tg,brand,vendor_article,number,price,client_type,date_status_order_done,date_status_item_returned,product_id,cost_price,shk,kbk_collect_delivery,manzana,percent_by_sku,manzana_for_item,price_wo_manzana,Товарное направление
1,006c9028-56fb-4a41-be10-6855473733ca,2024-05-28 16:08:46.847,Золотой пустотелый браслет Нонна с алмазной гр...,done,79922030205.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ01262494,6-01-010,3.68,3.68,NaN,585,НОННА ПУСТ,БРАСЛЕТ,КРАСНЫЙ,ЦБ БРАСЛЕТЫ,ADRIATICA,431-01-0080-30200,2405281608754,31278.00,MOBILE,2024-05-29,NaT,8378871,12182.40,2078609903845,6-01-010,5000,1.00,5000.00,26278.00,ЦБ
3,00d87b7f-954a-4e09-a249-a5231b78f28c,2024-05-29 12:19:05.967,Золотой пустотелый браслет Нонна с алмазной гр...,done,79122258992.00,mylnikova.tanya@internet.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ01262272,4-01-130,1.30,1.30,NaN,585,НОННА ПУСТ,БРАСЛЕТ,КРАСНЫЙ,ЦБ БРАСЛЕТЫ,ADRIATICA,411-01-0050-30200,2405291219934,12116.00,MOBILE,2024-05-29,NaT,1184881,4868.94,2078608734062,4-01-130,2423,1.00,2423.00,9693.00,ЦБ
6,02c0c006-57ae-441a-a477-7af182638acb,2024-05-29 10:35:56.175,Золотая цепочка Тондо,done,79027340707.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,fdbe3a06-9b1a-43cb-bc54-051a8ffb1692,TINKOFF_SBP,Тинькофф СБП,ТОВ0428306,2-13-001,2.28,2.28,NaN,585,ТОНДО,ЦЕПЬ,КРАСНЫЙ,ЦБ ЦЕПИ,NaN,"НЦ12-078 D0,2",2405291035248,16780.00,MOBILE,2024-05-29,NaT,2120566,7747.31,2078606345684,2-13-001,0,0.00,0.00,16780.00,ЦБ
7,02e71118-312e-44ca-9d1a-09a259bd40fb,2024-05-29 08:01:21.014,Золотое гладкое обручальное кольцо 3 мм,done,79641148662.00,masha-butterfly@bk.ru,done,PICKUP_NOW,YANDEX_KASSA,NaN,TINKOFF_SBP,Тинькофф СБП,ТОВ0832948,3-09-024,1.73,1.73,NaN,585,ИФ ОБРУЧ ГЛАДКАЯ,КОЛЬЦО ОБРУЧАЛЬНОЕ,КРАСНЫЙ,БК КОЛЬЦА ОБРУЧ,NaN,КО 03-00#ТД,2405290801327,13637.00,MOBILE,2024-05-29,NaT,2002928,5478.26,2078609547053,3-09-024,2727,1.00,2727.00,10910.00,БК
10,045e018f-9b7e-4b15-926e-b39f04f97e40,2024-05-29 04:26:30.424,Золотой пустотелый браслет Нонна с алмазной гр...,done,79827806313.00,NaN,done,PICKUP_NOW,YANDEX_KASSA,NaN,SBER,[YANDEX_KASSA] SBER - APP,ТОВ01262166,4-02-033,0.99,0.99,NaN,585,НОННА ПУСТ,БРАСЛЕТ,КРАСНЫЙ,ЦБ БРАСЛЕТЫ,ADRIATICA,411-01-0040-30200,2405290426874,7254.00,MOBILE,2024-05-29,NaT,1590230,2293.87,2078604828813,4-02-033,1450,1.00,1450.00,5804.00,ЦБ


In [24]:
data_today['date_status_order_done'].unique()

<DatetimeArray>
['2024-05-29 00:00:00']
Length: 1, dtype: datetime64[ns]

In [25]:
data_today.info()

<class 'pandas.core.frame.DataFrame'>
Index: 89 entries, 1 to 533
Data columns (total 38 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         89 non-null     object        
 1   created                    89 non-null     datetime64[ns]
 2   name                       89 non-null     object        
 3   state_order                89 non-null     object        
 4   recipient_phone            89 non-null     float64       
 5   recipient_email            23 non-null     object        
 6   state_item                 89 non-null     object        
 7   delivery_type              89 non-null     object        
 8   payment_type               89 non-null     object        
 9   employee_id                4 non-null      object        
 10  type_ibox                  89 non-null     object        
 11  type_name                  89 non-null     object        
 12  tov           

In [26]:
data_today['цена за гр'] = data_today['price_wo_manzana'] / data_today['avg_weight']
data_today['цена за гр'].describe()

count      89.00
mean     6825.91
std      1330.49
min      4171.52
25%      6097.30
50%      6828.12
75%      7546.76
max     14251.75
Name: цена за гр, dtype: float64

In [27]:
data_today['avg_weight'].describe()

count   89.00
mean     2.28
std      1.69
min      0.28
25%      1.21
50%      2.07
75%      2.71
max     11.22
Name: avg_weight, dtype: float64

In [28]:
data_today = data_today[~data_today['tg'].str.contains('ОПТ БК|БК ПЕЧАТКИ|ОПТ КОНГО')]

In [29]:
data_today = data_today[~data_today['client_type'].str.contains('OZON')]

In [30]:
print('Средняя цена продажи еком: ', (data_today['price_wo_manzana'].sum()/data_today['avg_weight'].sum()).round(0))

Средняя цена продажи еком:  6663.0


In [31]:
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')

In [32]:
data_today.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\ecom-sales\online_sales_{date_now}.xlsx', index=False)

In [33]:
tn = data_today.groupby('tg', as_index=False)[['price_wo_manzana', 'avg_weight', 'cost_price']].agg('sum')
tn['price_gramm'] = (tn['price_wo_manzana'] / tn['avg_weight']).round(0)

In [34]:
tn['price_gramm'] = tn['price_gramm'].astype('int')

In [35]:
tn['Наценка продажи онлайн'] =  tn['price_wo_manzana'] / tn['cost_price'] - 1

In [36]:
tn

,tg,price_wo_manzana,avg_weight,cost_price,price_gramm,Наценка продажи онлайн
0,БК КОЛЬЦА,12844.00,2.07,5596.12,6205,1.30
1,БК КОЛЬЦА ОБРУЧ,428116.00,69.79,195998.01,6134,1.18
2,БК ПОДВЕС ДЕКОР,21810.00,4.04,9405.83,5399,1.32
3,БК ПОДВЕС КУЛЬТ,93582.00,12.29,37393.98,7614,1.50
4,БК СЕРЬГИ,67009.00,8.81,25449.54,7606,1.63
5,ЦБ БРАСЛЕТЫ,384212.00,56.47,194508.02,6804,0.98
6,ЦБ ЦЕПИ,342020.00,49.08,149677.14,6969,1.29


In [37]:
print('Средняя цена продажи еком: ', (tn['price_wo_manzana'].sum() / tn['avg_weight'].sum()).round(0))

Средняя цена продажи еком:  6663.0


In [38]:
tn.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\ecom-sales\ecom_sales_tg\\ecom_sales_{date_now}.xlsx', index=False)

In [39]:
tn = tn.drop(columns=['price_wo_manzana', 'avg_weight', 'cost_price', 'Наценка продажи онлайн'])

In [40]:
tn.columns = ['tg', '585*Золотой онлайн продажи']

In [41]:
day

30

In [42]:
pd.read_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx').merge(tn, how = 'left', on = 'tg').to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx', index=False)